In [89]:
import requests
from bs4 import BeautifulSoup
import os
import json
import sys
sys.path.append("/Users/kuramochiosuke/.pyenv/versions/3.10.4/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages")
from requests_oauthlib import OAuth1Session
sys.path.append("/Users/kuramochiosuke/.pyenv/versions/3.10.4/lib/python3.10/site-packages")
from dotenv import find_dotenv, load_dotenv
import requests
import schedule
import time
import openpyxl


# .envファイルを探して読み込み
env_file = find_dotenv()
load_dotenv(env_file)  

CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_KEY = os.environ.get('ACCESS_KEY')
ACCESS_KEY_SECRET = os.environ.get('ACCESS_KEY_SECRET')

# Twitterの認証
twitter = OAuth1Session(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_KEY_SECRET)
print(twitter)
#エンドポイント
url_text = 'https://api.twitter.com/1.1/statuses/update.json'
url_media = "https://upload.twitter.com/1.1/media/upload.json"
print("投稿準備完了")

投稿準備完了


In [90]:
#nbaのページの指定
URL = 'https://basketballking.jp/news/category/world/nba'
# リクエストヘッダの指定
headers = {"User-Agent": "hoge"}
response = requests.get(URL,  headers=headers)
r_text=response.text
soup = BeautifulSoup(r_text, 'html.parser')

In [91]:
soup_a=soup.find_all("div",attrs={"class","container"})[1]
soup_article=soup_a.find_all("div",attrs={"class","news-category-list__vertical"})[0]
soup_article

<div class="news-category-list__vertical">
<a class="news-category-list__link" href="https://basketballking.jp/news/world/nba/20220704/381464.html?cx_cat=page1">
<div class="news-category-list__vertical__img">
<img alt="" class="attachment-350n-post-thumbnail size-350n-post-thumbnail wp-post-image" height="263" loading="lazy" sizes="(max-width: 350px) 100vw, 350px" src="https://basketballking.jp/wp-content/uploads/2022/07/a5f85a98c0940839a3de9eb9d4a3f865-350x263.jpg" srcset="https://basketballking.jp/wp-content/uploads/2022/07/a5f85a98c0940839a3de9eb9d4a3f865-350x263.jpg 350w, https://basketballking.jp/wp-content/uploads/2022/07/a5f85a98c0940839a3de9eb9d4a3f865-300x225.jpg 300w, https://basketballking.jp/wp-content/uploads/2022/07/a5f85a98c0940839a3de9eb9d4a3f865-768x576.jpg 768w, https://basketballking.jp/wp-content/uploads/2022/07/a5f85a98c0940839a3de9eb9d4a3f865-200x150.jpg 200w, https://basketballking.jp/wp-content/uploads/2022/07/a5f85a98c0940839a3de9eb9d4a3f865-500x375.jpg 500w, 

In [92]:
soup_title=soup_article.find_all("div",attrs={"class","news-category-list__vertical__title"})[0].text.replace("\n","").replace("\t","")
soup_title

'NBA全球団の直近ドラフト1位指名選手…一時代を築いた選手やハズレ指名も？'

In [93]:
soup_url=soup_article.find_all("a")[0]['href']
soup_url

'https://basketballking.jp/news/world/nba/20220704/381464.html?cx_cat=page1'

In [94]:
soup_img=soup_article.find("img")["src"]
soup_img

'https://basketballking.jp/wp-content/uploads/2022/07/a5f85a98c0940839a3de9eb9d4a3f865-350x263.jpg'

In [95]:
response = requests.get(soup_img)
image = response.content
files = {"media" : image}
req_media = twitter.post(url_media, files = files)
media_id = json.loads(req_media.text)['media_id']
print('画像の取得完了')

画像の取得完了


In [97]:
params = {'status':"NBA情報!!!\n\n{}\n\n記事↓↓↓\n{}".format(soup_title,soup_url),'media_ids':[media_id]}
wb = openpyxl.load_workbook('nba.xlsx')
time.sleep(1)
ws = wb["Sheet1"]
for i in range(wb['Sheet1'].max_row):
    if ws.cell(row=i+1,column=1).value==params["status"]:
        print("投稿済みです")
        break    
    elif i==wb['Sheet1'].max_row-1:   
        ws.cell(row=wb['Sheet1'].max_row+1,column=1).value = params["status"]
        wb.save('nba.xlsx')
        print("保存しました")
        twitter.post(url_text, params = params)
        print("投稿しました") 
print("処理終了しました")
print("")

投稿済みです
処理終了しました

